In [1]:
# %apt-get install -y python3-dev swig
# %pip install box2d-py

## Imports

In [2]:
# Envs
from collections import deque
import cv2
import imageio

## LunarLander
import gymnasium
import math

## CarRacing

# Network
import torch
import torch.nn as nn

# ReplayBuffer
import numpy as np

# MuZero
import os
from datetime import datetime
import matplotlib.pyplot as plt
import time

# Part 1

## Config

In [3]:
class Config:
    # Save graphs/anims
    GRAPH_DIR = "data/graphs/"
    ANIM_DIR = "data/anims/"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ### Game
    nb_episodes = 1000
    nb_test_chunk = 3
    nb_ep_bw_test = 20
    update_graph_interval = 5

    action_space = list(range(4))
    resolution = (96, 96)
    nb_channels = 1  # Number of channels in the observations, 1 for grayscale, 3 RGB
    stacked_observations = (
        4  # Number of previous observations and previous actions to add to the current observation
    )
    interval = 5  # Time-jumps between frames

    ### Self-Play
    max_moves = 400
    num_simulations = 30  # Number of future moves self-simulated
    discount = 0.999  # Chronological discount of the reward
    value_loss_weight = 0.25  # paper recommends 0.25

    # Root prior exploration noise
    root_dirichlet_alpha = 0.25
    root_exploration_fraction = 0.25

    # UCB formula
    pb_c_base = 19652
    pb_c_init = 1.25

    ### Network
    support_size = 20

    # Fully Connected Network
    encoding_size = 10
    fc_representation_layers = []
    fc_dynamics_layers = [64]
    fc_reward_layers = [64]
    fc_value_layers = [64]
    fc_policy_layers = [64]

    ### Training
    # Exponential learning rate schedule
    lr_init = 0.005
    lr_decay_rate = 1

    training_steps = 20000
    batch_size = 64

    ### Replay Buffer
    replay_buffer_size = 128  # Number of self-play games to keep in the replay buffer
    num_unroll_steps = 10  # Number of game moves to keep for every batch element
    td_steps = (
        15  # Number of steps in the future to take into account for calculating the target value
    )

    visit_softmax_temperature = 0.35

## Game

#### Lunar Lander

In [4]:
class LunarLander:
    def __init__(self, normalize=True, record=False):
        self.config = Config
        self.normalize = normalize
        self.record = record

        self.env = gymnasium.make("LunarLander-v3", render_mode="rgb_array")
        self.action_space = self.env.action_space

        self.frames = []
        self.observations = deque(maxlen=self.config.stacked_observations)

        self.step_count = 0
        self.total_reward = 0

    def step(self, action):
        observation, reward, terminated, truncated, _ = self.env.step(action)
        done = terminated or truncated

        frame = np.array(self.render())
        if self.record:
            self.frames.append(frame)

        observation = self._process_frame(frame)
        self.observations.append(observation)

        step_obs = torch.cat(list(self.observations), dim=0).unsqueeze(0)

        self.step_count += 1
        self.total_reward += reward
        if self.step_count >= self.config.max_moves:
            done = True

        return step_obs, reward, done

    def _process_frame(self, observation):
        target_shape = (
            self.config.resolution[0],
            self.config.resolution[1],
            self.config.nb_channels,
        )
        observation = cv2.resize(observation, target_shape[:2], interpolation=cv2.INTER_AREA)

        # observation = np.transpose(observation, (2, 0, 1))

        if self.config.nb_channels == 1:
            observation = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
            # observation = cv2.cvtColor(observation.transpose(1, 2, 0), cv2.COLOR_RGB2GRAY)
            observation = np.expand_dims(observation, axis=0)

        if self.normalize:
            observation = observation / 255.0

        return torch.from_numpy(observation).to(torch.float32)

    def render(self):
        return self.env.render()

    def reset(self):
        self.frames = []
        self.observations = deque(maxlen=self.config.stacked_observations)

        self.step_count = 0
        self.total_reward = 0

        self.env.reset()
        observation = np.array(self.env.render())

        processed_obs = self._process_frame(observation)
        for _ in range(self.config.stacked_observations):
            self.observations.append(processed_obs)

        step_obs = torch.cat(list(self.observations), dim=0).unsqueeze(0)

        return step_obs

    def close(self):
        self.env.close()

    def save_video(self, VID_DIR, episode, fps=60, test_ep=None, prnt=True):
        # Save the frames as a video
        if test_ep is None:
            video_filename = f"{VID_DIR}/ep_{episode}.mp4"
        else:
            video_filename = f"{VID_DIR}/ep_{episode}_test_{test_ep}.mp4"
        if prnt:
            print(f"Saving video to {video_filename}")
        imageio.mimsave(video_filename, self.frames, fps=fps)

#### Car Racing

In [5]:
class CarRacing:
    def __init__(self, normalize=True, record=False):
        self.config = Config
        self.normalize = normalize
        self.record = record

        self.env = gymnasium.make(
            "CarRacing-v2", render_mode="rgb_array", domain_randomize=False, continuous=False
        )
        self.action_space = self.env.action_space

        self.frames = []
        self.observations = deque(maxlen=self.config.stacked_observations)

        self.step_count = 0
        self.total_reward = 0

    def step(self, action):
        observation, reward, terminated, truncated, _ = self.env.step(action)
        if self.record:
            self.frames.append(np.array(self.render()))

        observation = self._process_frame(observation)
        self.observations.append(observation)

        step_obs = torch.cat(list(self.observations), dim=0).unsqueeze(0)

        done = terminated or truncated
        if self.step_count >= self.config.max_moves:
            done = True

        self.step_count += 1
        self.total_reward += reward

        return step_obs, reward, done

    def _process_frame(self, observation):
        observation = np.transpose(observation, (2, 0, 1))

        if self.config.nb_channels == 1:
            observation = cv2.cvtColor(observation.transpose(1, 2, 0), cv2.COLOR_RGB2GRAY)
            observation = np.expand_dims(observation, axis=0)

        if self.normalize:
            observation = observation / 255.0

        return torch.from_numpy(observation).to(torch.float32)

    def render(self):
        return self.env.render()

    def reset(self):
        self.frames = []
        self.observations = deque(maxlen=self.config.stacked_observations)

        self.step_count = 0
        self.total_reward = 0

        observation, _ = self.env.reset()
        if self.record:
            self.frames.append(np.array(self.render()))

        observation = self._process_frame(observation)
        for _ in range(self.config.stacked_observations):
            self.observations.append(observation)

        step_obs = torch.cat(list(self.observations), dim=0).unsqueeze(0)

        return step_obs

    def close(self):
        self.env.close()

    def save_video(self, VID_DIR, episode, fps=60, test_ep=None, prnt=True):
        # Save the frames as a video
        if test_ep is None:
            video_filename = f"{VID_DIR}/ep_{episode}.mp4"
        else:
            video_filename = f"{VID_DIR}/ep_{episode}_test_{test_ep}.mp4"
        if prnt:
            print(f"Saving video to {video_filename}")
        imageio.mimsave(video_filename, self.frames, fps=fps)

## Models

In [6]:
class MuZeroNetwork(nn.Module):
    def __init__(self):
        super(MuZeroNetwork, self).__init__()

        self.config = Config
        self.device = self.config.device
        self.to(self.device)

        self.fc_reward_layers = Config.fc_reward_layers
        self.fc_value_layers = Config.fc_value_layers
        self.fc_policy_layers = Config.fc_policy_layers
        self.fc_representation_layers = Config.fc_representation_layers
        self.fc_dynamics_layers = Config.fc_dynamics_layers
        self.full_support_size = 2 * self.config.support_size + 1
        self.support_values = (
            torch.arange(
                -self.config.support_size, self.config.support_size + 1, dtype=torch.float32
            )
            .unsqueeze(0)
            .to(self.device)
        )

        self.repr_conv = nn.Sequential(
            nn.Conv2d(self.config.nb_channels * self.config.stacked_observations, 32, 8, stride=4),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.Flatten(),
        )
        num_features = len(
            self.repr_conv(
                torch.zeros(
                    (
                        1,
                        self.config.nb_channels * self.config.stacked_observations,
                        self.config.resolution[0],
                        self.config.resolution[1],
                    )
                )
            )[0, :]
        )
        self.fc = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.Linear(128, self.config.encoding_size),
        )

        self.representation_network = nn.Sequential(
            *self.repr_conv,
            *self.fc,
        )

        self.dynamics_encoded_state_network = network_builder(
            self.config.encoding_size + len(self.config.action_space),
            self.fc_dynamics_layers,
            self.config.encoding_size,
        )
        self.dynamics_reward_network = network_builder(
            self.config.encoding_size, self.config.fc_reward_layers, self.full_support_size
        )
        self.prediction_policy_network = network_builder(
            self.config.encoding_size, self.fc_policy_layers, len(self.config.action_space)
        )
        self.prediction_value_network = network_builder(
            self.config.encoding_size, self.fc_value_layers, self.full_support_size
        )

        self.optimizer = torch.optim.Adam(
            self.parameters(),
            lr=Config.lr_init,
            weight_decay=Config.lr_decay_rate,
        )

    def representation_function(self, observation):
        encoded_state = self.representation_network(observation)

        # Scale encoded state between [0, 1]
        min_encoded_state = encoded_state.min(1, keepdim=True)[0]
        max_encoded_state = encoded_state.max(1, keepdim=True)[0]
        scale_encoded_state = max_encoded_state - min_encoded_state
        scale_encoded_state[scale_encoded_state < 1e-5] += 1e-5
        encoded_state_normalized = (encoded_state - min_encoded_state) / scale_encoded_state
        return encoded_state_normalized

    def prediction_function(self, encoded_state):
        policy_logits = self.prediction_policy_network(encoded_state)
        value = self.prediction_value_network(encoded_state)
        return policy_logits, value

    def dynamics_function(self, encoded_state, action):
        # Stack encoded_state with a game specific one hot encoded action
        action_one_hot = torch.zeros(
            (action.shape[0], len(self.config.action_space)), device=self.device
        ).float()
        action_one_hot.scatter_(1, action.long(), 1.0)

        x = torch.cat((encoded_state, action_one_hot), dim=1)

        next_encoded_state = self.dynamics_encoded_state_network(x)

        reward = self.dynamics_reward_network(next_encoded_state)

        # Scale encoded state between [0, 1]
        min_next_encoded_state = next_encoded_state.min(1, keepdim=True)[0]
        max_next_encoded_state = next_encoded_state.max(1, keepdim=True)[0]
        scale_next_encoded_state = max_next_encoded_state - min_next_encoded_state
        scale_next_encoded_state[scale_next_encoded_state < 1e-5] += 1e-5
        next_encoded_state_normalized = (
            next_encoded_state - min_next_encoded_state
        ) / scale_next_encoded_state

        return next_encoded_state_normalized, reward

    def initial_inference(self, observation):
        observation = observation.to(self.device)
        encoded_state = self.representation_function(observation)
        policy_logits, value = self.prediction_function(encoded_state)
        # reward equal to 0 for consistency
        reward = torch.log(
            (
                torch.zeros(1, self.full_support_size)
                .scatter(1, torch.tensor([[self.full_support_size // 2]]).long(), 1.0)
                .repeat(len(observation), 1)
                .to(self.device)
            )
        )
        return value, reward, policy_logits, encoded_state

    def recurrent_inference(self, encoded_state, action):
        encoded_state = encoded_state.to(self.config.device)
        action = action.to(self.config.device)
        next_encoded_state, reward = self.dynamics_function(encoded_state, action)
        policy_logits, value = self.prediction_function(next_encoded_state)
        return value, reward, policy_logits, next_encoded_state


def network_builder(
    input_size,
    layer_sizes,
    output_size,
    output_activation=torch.nn.Identity,
    activation=torch.nn.ELU,
    input_layers=[],
):
    sizes = [input_size] + layer_sizes + [output_size]
    layers = []
    layers.extend(input_layers)
    for i in range(len(sizes) - 1):
        act = activation if i < len(sizes) - 2 else output_activation
        layers += [torch.nn.Linear(sizes[i], sizes[i + 1]), act()]
    return torch.nn.Sequential(*layers)


def support_to_scalar(logits, support_size):
    """
    Transform a categorical representation to a scalar
    """
    # Decode to a scalar
    probabilities = torch.softmax(logits, dim=1)
    support = (
        torch.tensor([x for x in range(-support_size, support_size + 1)], device=logits.device)
        .expand(probabilities.shape)
        .float()
    )

    x = torch.sum(support * probabilities, dim=1, keepdim=True)

    return torch.sign(x) * (
        ((torch.sqrt(1 + 4 * 0.001 * (torch.abs(x) + 1 + 0.001)) - 1) / (2 * 0.001)) ** 2 - 1
    )


def scalar_to_support(x, support_size):
    """
    Transform a scalar to a categorical representation with (2 * support_size + 1) categories
    """
    # Reduce the scale (defined in https://arxiv.org/abs/1805.11593)
    x = torch.sign(x) * (torch.sqrt(torch.abs(x) + 1) - 1) + 0.001 * x

    # Encode on a vector
    x = torch.clamp(x, -support_size, support_size)
    floor = x.floor()
    prob = x - floor
    logits = torch.zeros(x.shape[0], x.shape[1], 2 * support_size + 1, device=x.device)
    logits.scatter_(2, (floor + support_size).long().unsqueeze(-1), (1 - prob).unsqueeze(-1))
    indexes = floor + support_size + 1
    prob = prob.masked_fill_(2 * support_size < indexes, 0.0)
    indexes = indexes.masked_fill_(2 * support_size < indexes, 0.0)
    logits.scatter_(2, indexes.long().unsqueeze(-1), prob.unsqueeze(-1))
    return logits

## Replay buffer

In [7]:
class ReplayBuffer:
    def __init__(self):
        """
        Initialize the replay buffer with a given size.
        """
        self.config = Config
        self.device = self.config.device

        self.buffer = deque(maxlen=self.config.replay_buffer_size)

        self.num_played_steps = 0
        self.num_played_games = 0
        self.total_samples = 0

    def __len__(self):
        return len(self.buffer)

    def save_game(self, game_history):
        self.buffer.append(game_history)
        self.num_played_games += 1
        self.num_played_steps += len(game_history.root_values)
        self.total_samples = sum(len(game.root_values) for game in self.buffer)

    def compute_target_value(self, game_history, index):
        # The value target is the discounted root value of the search tree td_steps into the future, plus the discounted sum of all rewards until then.
        bootstrap_index = index + self.config.td_steps
        if bootstrap_index < len(game_history.root_values):
            root_values = game_history.root_values
            last_step_value = root_values[bootstrap_index]

            value = last_step_value * self.config.discount**self.config.td_steps
        else:
            value = 0

        for i, reward in enumerate(game_history.reward_history[index + 1 : bootstrap_index + 1]):
            value += reward * self.config.discount**i

        return value

    def get_batch(self):
        index_batch = []
        observation_batch = []
        action_batch = []
        value_batch = []
        reward_batch = []
        policy_batch = []
        gradient_scale_batch = []

        for game_id, game_history in self.sample_game(self.config.batch_size):
            game_pos = self.sample_position(game_history)

            values, rewards, policies, actions = self.make_target(game_history, game_pos)

            observation_batch.append(game_history.observation_history[game_pos])

            index_batch.append([game_id, game_pos])
            action_batch.append(actions)
            value_batch.append(values)
            reward_batch.append(rewards)
            policy_batch.append(policies)
            gradient_scale_batch.append(
                [min(self.config.num_unroll_steps, len(game_history.action_history) - game_pos)]
                * len(actions)
            )

        # Move all tensors in observation_batch to the same device
        observation_batch = [obs.to(self.device) for obs in observation_batch]

        # Now stack the tensors
        observation_batch = torch.stack([obs.squeeze(0) for obs in observation_batch], axis=0)

        return (
            index_batch,
            (
                observation_batch,
                action_batch,
                value_batch,
                reward_batch,
                policy_batch,
                gradient_scale_batch,
            ),
        )

    def _generate_single_batch(self, game_history, game_id, game_pos):
        values, rewards, policies, actions, observations = self.make_target(game_history, game_pos)

        single_index = [game_id, game_pos]
        single_observation = observations
        single_action = actions
        single_value = values
        single_reward = rewards
        single_policy = policies
        single_gradient_scale = [
            min(self.config.num_unroll_steps, len(game_history.action_history) - game_pos)
        ] * len(actions)

        return (
            single_index,
            single_observation,
            single_action,
            single_value,
            single_reward,
            single_policy,
            single_gradient_scale,
        )

    def sample_game(self, n_games=1):
        """
        Sample n_games from the buffer uniformly.
        See paper appendix Training.
        """
        game_indices = np.random.choice(len(self.buffer), size=n_games, replace=True)
        game_ids = [
            self.num_played_games - len(self.buffer) + game_index for game_index in game_indices
        ]

        return [(game_index, self.buffer[game_index]) for game_index in game_indices]

    def sample_position(self, game_history):
        """
        Sample position from game uniformly.
        See paper appendix Training.
        """
        return np.random.choice(len(game_history.root_values))

    def make_target(self, game_history, state_index):
        """
        Generate targets for every unroll steps.
        """
        target_values = []
        target_rewards = []
        target_policies = []
        actions = []

        for current_index in range(state_index, state_index + self.config.num_unroll_steps + 1):
            value = self.compute_target_value(game_history, current_index)

            if current_index < len(game_history.root_values):
                target_values.append(value)
                target_rewards.append(game_history.reward_history[current_index])
                target_policies.append(game_history.child_visits[current_index])
                actions.append(game_history.action_history[current_index])
            elif current_index == len(game_history.root_values):
                target_values.append(0)
                target_rewards.append(game_history.reward_history[current_index])
                # Uniform policy
                target_policies.append(
                    [
                        1 / len(game_history.child_visits[0])
                        for _ in range(len(game_history.child_visits[0]))
                    ]
                )
                actions.append(game_history.action_history[current_index])
            else:
                # States past the end of games are treated as absorbing states
                target_values.append(0)
                target_rewards.append(0)
                # Uniform policy
                target_policies.append(
                    [
                        1 / len(game_history.child_visits[0])
                        for _ in range(len(game_history.child_visits[0]))
                    ]
                )
                actions.append(np.random.choice(self.config.action_space))

        return target_values, target_rewards, target_policies, actions

## GameHistory

In [8]:
class GameHistory:
    """
    Store only usefull information of a self-play game.
    """

    def __init__(self):
        self.observation_history = []
        self.action_history = []
        self.reward_history = []
        self.child_visits = []
        self.root_values = []

    def store_search_statistics(self, root, action_space):
        # Turn visit count from root into a policy
        if root is not None:
            sum_visits = sum(child.visit_count for child in root.children.values())
            self.child_visits.append(
                [
                    root.children[a].visit_count / sum_visits if a in root.children else 0
                    for a in action_space
                ]
            )

            self.root_values.append(root.value())
        else:
            self.root_values.append(None)

## MCTS

In [9]:
class Node:
    def __init__(self, prior):
        self.device = Config.device

        self.visit_count = 0
        self.prior = prior
        self.value_sum = 0
        self.children = {}
        self.hidden_state = None
        self.reward = 0

    def expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count

    def expand(self, actions, reward, policy_logits, hidden_state):
        """
        # FROM PAPER CODE GITHUB
        We expand a node using the value, reward and policy prediction obtained from the
        neural network.
        """
        self.reward = reward
        self.hidden_state = hidden_state

        # policy_values = torch.softmax(torch.tensor([policy_logits[0][a] for a in actions]), dim=0).tolist()
        policy_values = torch.softmax(
            torch.tensor([policy_logits[0][a] for a in actions]).to(self.device), dim=0
        ).tolist()
        policy = {a: policy_values[i] for i, a in enumerate(actions)}

        for action, p in policy.items():
            self.children[action] = Node(p)

    def add_exploration_noise(self, dirichlet_alpha, exploration_fraction):
        """
        At the start of each search, we add dirichlet noise to the prior of the root to
        encourage the search to explore new actions.
        """
        actions = list(self.children.keys())
        noise = np.random.dirichlet([dirichlet_alpha] * len(actions))
        frac = exploration_fraction
        for a, n in zip(actions, noise):
            self.children[a].prior = self.children[a].prior * (1 - frac) + n * frac


class MinMaxStats:
    """
    COPIED
    A class that holds the min-max values of the tree.
    """

    def __init__(self):
        self.maximum = -float("inf")
        self.minimum = float("inf")

    def update(self, value):
        self.maximum = max(self.maximum, value)
        self.minimum = min(self.minimum, value)

    def normalize(self, value):
        if self.maximum > self.minimum:
            # We normalize only when we have set the maximum and minimum values
            return (value - self.minimum) / (self.maximum - self.minimum)
        return value


def select_action(node, best=False):
    """
    Select action according to the visit count distribution and the temperature.
    The temperature is changed dynamically with the visit_softmax_temperature function
    in the config.
    """

    visit_counts = np.array([child.visit_count for child in node.children.values()], dtype="int32")
    actions = [action for action in node.children.keys()]

    visit_count_distribution = visit_counts ** (1 / Config.visit_softmax_temperature)
    visit_count_distribution = visit_count_distribution / sum(visit_count_distribution)

    if best:
        action = actions[np.argmax(visit_count_distribution)]
    else:
        action = np.random.choice(actions, p=visit_count_distribution)

    # if np.random.random() < 0.01:
    #     print(visit_count_distribution)
    #     print(action)

    return action


class MCTS:
    def __init__(self) -> None:
        self.config = Config

        self.discount = Config.discount

    def run(self, network: MuZeroNetwork, observation, add_exploration_noise=True):
        root = Node(0)

        root_predicted_value, reward, policy_logits, hidden_state = network.initial_inference(
            observation
        )

        root_predicted_value = support_to_scalar(
            root_predicted_value, self.config.support_size
        ).item()
        reward = support_to_scalar(reward, self.config.support_size).item()

        root.expand(
            self.config.action_space,
            reward,
            policy_logits,
            hidden_state,
        )

        if add_exploration_noise:
            root.add_exploration_noise(
                dirichlet_alpha=self.config.root_dirichlet_alpha,
                exploration_fraction=self.config.root_exploration_fraction,
            )

        min_max_stats = MinMaxStats()

        max_tree_depth = 0
        for _ in range(self.config.num_simulations):
            node = root
            search_path = [node]
            current_tree_depth = 0

            while node.expanded():
                current_tree_depth += 1
                action, node = self.select_child(node, min_max_stats)
                search_path.append(node)

            parent = search_path[-2]
            value, reward, policy_logits, hidden_state = network.recurrent_inference(
                parent.hidden_state, torch.tensor([[action]])
            )
            value = support_to_scalar(value, self.config.support_size).item()
            reward = support_to_scalar(reward, self.config.support_size).item()
            node.expand(
                self.config.action_space,
                reward,
                policy_logits,
                hidden_state,
            )

            self.backpropagate(search_path, value, min_max_stats)

            max_tree_depth = max(max_tree_depth, current_tree_depth)

        return root

    def select_child(self, node, min_max_stats):
        """
        COPIED FROM GITHUB
        Select the child with the highest UCB score.
        """
        max_ucb = max(
            self.ucb_score(node, child, min_max_stats) for action, child in node.children.items()
        )
        action = np.random.choice(
            [
                action
                for action, child in node.children.items()
                if self.ucb_score(node, child, min_max_stats) == max_ucb
            ]
        )
        return action, node.children[action]

    def ucb_score(self, parent, child, min_max_stats):
        """
        COPIED FROM GITHUB
        The score for a node is based on its value, plus an exploration bonus based on the prior.
        """
        pb_c = (
            math.log((parent.visit_count + self.config.pb_c_base + 1) / self.config.pb_c_base)
            + self.config.pb_c_init
        )
        pb_c *= math.sqrt(parent.visit_count) / (child.visit_count + 1)

        prior_score = pb_c * child.prior

        if child.visit_count > 0:
            # Mean value Q
            value_score = min_max_stats.normalize(child.reward + self.discount * child.value())
        else:
            value_score = 0

        return prior_score + value_score

    def backpropagate(self, search_path, value, min_max_stats):
        """
        At the end of a simulation, we propagate the evaluation all the way up the tree
        to the root.
        """
        for node in reversed(search_path):
            node.value_sum += value
            node.visit_count += 1
            min_max_stats.update(node.reward + self.discount * node.value())

            value = node.reward + self.discount * value

## MuZero

In [10]:
class MuZero:
    def __init__(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            torch.manual_seed(seed)

        self.config = Config
        self.device = self.config.device

        self.network = MuZeroNetwork()
        self.network.to(self.device)

        # self.env = CarRacing()
        self.env = LunarLander()
        self.buffer = ReplayBuffer()

        self.VID_DIR = f'{Config.ANIM_DIR}/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'
        self.GRAPH_DIR = f'{Config.GRAPH_DIR}/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'

        if not os.path.exists(f"{self.VID_DIR}/test"):
            os.makedirs(f"{self.VID_DIR}/test")
        if not os.path.exists(f"{self.GRAPH_DIR}"):
            os.makedirs(f"{self.GRAPH_DIR}")

        self.training_step = 0
        self.training_history = {
            "loss": [],
            "value_loss": [],
            "reward_loss": [],
            "policy_loss": [],
            "reward": [],
        }

    def train(self):
        for episode in range(self.config.nb_episodes):
            # Test the MuZero network
            if episode % self.config.nb_ep_bw_test == 0:
                self.test_play(episode)

            # Run self-play simulation
            game_history = self.play(episode)
            self.training_history["reward"].append(self.env.total_reward)

            # Add trajectory data to replay buffer
            self.buffer.save_game(game_history)

            # Train the MuZero network
            if self.training_step < self.config.training_steps:
                batch_id, batch = self.buffer.get_batch()

                (
                    total_loss,
                    value_loss,
                    reward_loss,
                    policy_loss,
                ) = self.update_weights(batch)

                self.training_history["loss"].append(total_loss)
                self.training_history["value_loss"].append(value_loss)
                self.training_history["reward_loss"].append(reward_loss)
                self.training_history["policy_loss"].append(policy_loss)

                print(
                    f"\nLoss: {total_loss:.4f} | Value loss: {value_loss:.4f} | Reward loss: {reward_loss:.4f} | Policy loss: {policy_loss:.4f}"
                )

            # Update graphs every few episodes
            if episode % self.config.update_graph_interval == 0:
                self.update_graphs()

        print("End of training. Last tests...\n")

        self.test_play(episode)
        self.env.close()

    def play(self, episode: int):
        start = time.time()

        print(f"\n\nTraining episode {episode}...")

        # Train the MuZero network
        observation = self.env.reset()
        observation = observation.to(self.device)

        done = False

        game_history = GameHistory()
        game_history.action_history.append(0)
        game_history.observation_history.append(observation)
        game_history.reward_history.append(0)

        # Interaction loop
        while not done:
            # Plan and act using Monte-Carlo Tree Search (MCTS)
            root = MCTS().run(self.network, observation)
            action = select_action(root)

            # Interact with the environment
            observation, reward, done = self.env.step(action)

            game_history.store_search_statistics(root, self.config.action_space)
            game_history.action_history.append(action)
            game_history.observation_history.append(observation)
            game_history.reward_history.append(reward)

        print(
            f"Total reward: {self.env.total_reward:.2f} | Steps: {self.env.step_count} | Time: {time.time() - start:.2f}s"
        )

        return game_history

    def test_play(self, episode: int):
        print(f"\n\nTesting episode {episode}...")

        self.env.record = True

        # Test the MuZero network
        for test_ep in range(self.config.nb_test_chunk):
            # Reset the environment
            test_observation = self.env.reset()
            test_observation.to(self.device)
            test_done = False

            # Interaction loop for the test episode
            while not test_done:
                # Plan and act using Monte-Carlo Tree Search (MCTS)
                test_root = MCTS().run(self.network, test_observation, add_exploration_noise=False)
                test_action = select_action(test_root)  # , best=True)

                test_observation, test_reward, test_done = self.env.step(test_action)

            self.env.save_video(self.VID_DIR, episode, test_ep=test_ep)
            print(f"Total reward: {self.env.total_reward:.2f}")

        self.env.record = False

    def update_graphs(self):
        plt.figure(figsize=(12, 8))
        plt.plot(self.training_history["loss"], label="Total loss")
        plt.plot(self.training_history["value_loss"], label="Value loss")
        plt.plot(self.training_history["reward_loss"], label="Reward loss")
        plt.plot(self.training_history["policy_loss"], label="Policy loss")
        plt.legend()
        plt.savefig(f"{self.GRAPH_DIR}/loss.png")
        plt.close()

        smoothed_rewards = [self.training_history["reward"][0]]

        for i in range(1, len(self.training_history["reward"])):
            start = max(0, i - 6)
            mean_reward = sum(self.training_history["reward"][start : i + 1]) / (i - start + 1)
            smoothed_rewards.append(mean_reward)

        plt.figure(figsize=(12, 8))
        # plt.plot(self.training_history["reward"], label="Reward")
        plt.plot(smoothed_rewards, label="Smoothed reward")
        plt.legend()
        plt.savefig(f"{self.GRAPH_DIR}/reward.png")
        plt.close()

    def update_weights(self, batch):
        """
        Perform one training step.
        """
        (
            observation_batch,
            action_batch,
            target_value,
            target_reward,
            target_policy,
            gradient_scale_batch,
        ) = batch

        observation_batch = observation_batch.clone().detach()
        action_batch = torch.tensor(action_batch).to(self.device).long().unsqueeze(-1)
        target_value = torch.tensor(target_value).to(self.device).float()
        target_reward = torch.tensor(target_reward).to(self.device).float()
        target_policy = torch.tensor(target_policy).to(self.device).float()
        gradient_scale_batch = torch.tensor(gradient_scale_batch).to(self.device).float()

        target_value = scalar_to_support(target_value, self.config.support_size)
        target_reward = scalar_to_support(target_reward, self.config.support_size)

        ## Generate predictions
        value, reward, policy_logits, hidden_state = self.network.initial_inference(
            observation_batch
        )

        predictions = [(value, reward, policy_logits)]
        for i in range(1, action_batch.shape[1]):
            value, reward, policy_logits, hidden_state = self.network.recurrent_inference(
                hidden_state, action_batch[:, i]
            )

            # Scale the gradient at the start of the dynamics function (See paper appendix Training)

            # value = support_to_scalar(value, self.config.support_size).item()
            # reward = support_to_scalar(reward, self.config.support_size).item()

            hidden_state.register_hook(lambda grad: grad * 0.5)
            predictions.append((value, reward, policy_logits))

        ## Compute losses
        value_loss, reward_loss, policy_loss = (0, 0, 0)
        value, reward, policy_logits = predictions[0]
        # Ignore reward loss for the first batch step
        current_value_loss, _, current_policy_loss = self.loss_function(
            value.squeeze(-1),
            reward.squeeze(-1),
            policy_logits,
            target_value[:, 0],
            target_reward[:, 0],
            target_policy[:, 0],
        )
        value_loss += current_value_loss
        policy_loss += current_policy_loss

        for i in range(1, len(predictions)):
            value, reward, policy_logits = predictions[i]
            # print(f"Value: {value.squeeze(-1).shape}")
            # print(f"Reward: {reward.squeeze(-1).shape}")
            # print(f"Policy logits: {policy_logits.shape}")
            # print(f"Target value: {target_value[:, i].shape}")
            # print(f"Target reward: {target_reward[:, i].shape}")
            # print(f"Target policy: {target_policy[:, i].shape}")

            # print(support_to_scalar(value.squeeze(-1)[0], 41))
            # print(support_to_scalar(target_value[:, i][0], 41))

            # raise Exception

            (
                current_value_loss,
                current_reward_loss,
                current_policy_loss,
            ) = self.loss_function(
                value.squeeze(-1),
                reward.squeeze(-1),
                policy_logits,
                target_value[:, i],
                target_reward[:, i],
                target_policy[:, i],
            )

            # Scale gradient by the number of unroll steps (See paper appendix Training)
            current_value_loss.register_hook(lambda grad: grad / gradient_scale_batch[:, i])
            current_reward_loss.register_hook(lambda grad: grad / gradient_scale_batch[:, i])
            current_policy_loss.register_hook(lambda grad: grad / gradient_scale_batch[:, i])

            value_loss += current_value_loss
            reward_loss += current_reward_loss
            policy_loss += current_policy_loss

        # value_loss *= 0
        # reward_loss *= 0
        # policy_loss *= 1

        loss = value_loss * self.config.value_loss_weight + reward_loss + policy_loss

        # Mean over batch dimension (pseudocode do a sum)
        loss = loss.mean()

        # Optimize
        self.network.optimizer.zero_grad()
        loss.backward()
        self.network.optimizer.step()
        self.training_step += 1

        return (
            # For log purpose
            loss.item(),
            value_loss.mean().item(),
            reward_loss.mean().item(),
            policy_loss.mean().item(),
        )

    @staticmethod
    def loss_function(
        value,
        reward,
        policy_logits,
        target_value,
        target_reward,
        target_policy,
    ):
        value_loss = torch.sum(-target_value * torch.nn.LogSoftmax(dim=1)(value), dim=1)
        reward_loss = torch.sum(-target_reward * torch.nn.LogSoftmax(dim=1)(reward), dim=1)
        policy_loss = torch.sum(-target_policy * torch.nn.LogSoftmax(dim=1)(policy_logits), dim=1)

        return value_loss, reward_loss, policy_loss

# Part 2

## Run

In [11]:
muzero = MuZero()

muzero.train()



Testing episode 0...
Saving video to data/anims//2024-10-18_15-47-50/ep_0_test_0.mp4


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Total reward: -237.97


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saving video to data/anims//2024-10-18_15-47-50/ep_0_test_1.mp4
Total reward: -88.16


...

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saving video to data/anims//2024-10-18_15-47-50/ep_500_test_1.mp4
Total reward: -143.65


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saving video to data/anims//2024-10-18_15-47-50/ep_500_test_2.mp4
Total reward: -302.95


Training episode 500...
Total reward: -147.98 | Steps: 115 | Time: 1.90s

Loss: 58.9543 | Value loss: 40.5315 | Reward loss: 33.5721 | Policy loss: 15.2493


Training episode 501...
Total reward: -102.02 | Steps: 76 | Time: 1.27s

Loss: 58.9086 | Value loss: 40.5433 | Reward loss: 33.5235 | Policy loss: 15.2492


Training episode 502...
Total reward: -113.02 | Steps: 85 | Time: 1.30s

Loss: 58.9777 | Value loss: 40.5433 | Reward loss: 33.5927 | Policy loss: 15.2492


Training episode 503...
Total reward: -223.39 | Steps: 72 | Time: 1.16s

Loss: 58.9792 | Value loss: 40.5307 | Reward loss: 33.5972 | Policy loss: 15.2493


Training episode 504...


KeyboardInterrupt: 